In [1]:
# conda - claimaduit

In [18]:
# import dependencies
import pandas as pd
import mysql.connector as mc
import pdfkit as pdf
import os
import numpy as np

from config import vgc_host, vgc_u, vgc_pw, mysql_host, mysql_u, mysql_pw

In [3]:
# MySQL
def mysql_q (u, p, h, db, sql, cols, commit):
    # cols (0 = no, 1 = yes)
    # commit (select = 0, insert/update = 1)

    # connect to db
    if db == 'MULTI':
        cnx = mc.connect(user=u, password=p, host=h) 
    else:      
        cnx = mc.connect(user=u, password=p, host=h, database=db)
        
    cursor = cnx.cursor()

    # commit?
    if commit == 1:
        cursor.execute(sql)
        cnx.commit()
        sql_result = 0     
    else:
        cursor.execute(sql)
        sql_result = cursor.fetchall()

    # columns ?
    if cols == 1:
        columns=list([x[0] for x in cursor.description])
        # close connection
        cursor.close()
        cnx.close()
        # return query result [0] and columns [1]
        return sql_result, columns
    else:
        # close connection
        cursor.close()
        cnx.close()
        # return query result
        return sql_result

In [4]:
# set date range
begDate = '2022-06-06'
endDate = '2022-06-10'

In [5]:
# get adjuster limits
adj_limits_sql = '''
                    SELECT adjuster, authority, effective_date
                    FROM authority;'''

adj_limits = mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_audit', adj_limits_sql, 1, 0)
# create df from query results
adj_limits_df = pd.DataFrame(adj_limits[0])
# get column names from results
adj_limits_df.columns = adj_limits[1]

In [6]:
# get all paid claims
claims_sql = '''
                SELECT c.claim_id, c.claim_nbr, c.paid_amount, c.paid_date, s.update_by, MAX(s.status_id)
                FROM claims c
                INNER JOIN claim_status s
                    USING(claim_id)
                WHERE c.paid_amount > 0
                    AND s.status_desc_id = 8
                    AND c.paid_date BETWEEN '{BegDate}' AND '{EndDate}'
                    AND s.update_by <> 'JARED BEHLER'
                GROUP BY c.claim_id;           
            '''.format(BegDate=begDate, EndDate=endDate)

In [7]:
# run sql query
claims = mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', claims_sql, 1, 0)
# create df from query results
claims_df = pd.DataFrame(claims[0])
# get column names from results
claims_df.columns = claims[1]
claims_df.rename(columns = {'update_by':'adjuster'}, inplace = True)
# claims_df['authority'] = 0

In [8]:
# create audit_df for results
# audit_df = pd.DataFrame(columns = ['claim_id', 'claim_nbr', 'paid_amount', 'paid_date', 'update_by', 'authority'])
audit_columns = ['claim_id', 'claim_nbr', 'paid_amount', 'paid_date', 'adjuster', 'authority']
audit_df = pd.DataFrame(columns = audit_columns)

Original Method

In [9]:
# # get claims within each processors limit
# for index, row in adj_limits_df.iterrows():
#     # filter claims_df by adj_limits_df
#     filtered_df = claims_df[(claims_df['adjuster'] == row['adjuster']) &
#                             (claims_df['paid_amount'] < row['authority'])]
#     # Random sample of results
#     # If less than 5 then select all
#     if filtered_df.shape[0] < 5:
#         random_df = filtered_df
#     else:
#         random_df = filtered_df.sample(n=5, random_state=13)
#     # Add column to filtered_df with adjuster limit
#     random_df['authority'] = row['authority']
#     # Remove column 'MAX(s.status_id)' 
#     random_df.drop(columns=['MAX(s.status_id)'], inplace=True)
#     # add random sample to audit_df
#     audit_df = pd.concat([audit_df, random_df])

New Method

In [10]:
claims_df.head(5)

,claim_id,claim_nbr,paid_amount,paid_date,adjuster,MAX(s.status_id)
0,74736,2022033075741,752.37,2022-06-07,CRISTAL WILLIAMS,405896
1,74821,2022041275826,1243.25,2022-06-06,Leah Shepherd,405858
2,74850,2022041275855,930.03,2022-06-10,CRISTAL WILLIAMS,406079
3,75177,2022052076182,5327.77,2022-06-09,Leah Shepherd,406008
4,75236,2022052776241,1217.78,2022-06-10,Leah Shepherd,406102


In [12]:
adj_limits_df

,adjuster,authority,effective_date
0,Leah Shepherd,1000.0,2022-06-06
1,KOLLEEN MCDONNOLD,1000.0,2022-06-06
2,CRISTAL WILLIAMS,1000.0,2022-06-06
3,Leah Shepherd,10000.0,2022-06-09


In [26]:
adjusters = np.unique(adj_limits_df['adjuster'].tolist())
print(adjusters)

['CRISTAL WILLIAMS' 'KOLLEEN MCDONNOLD' 'Leah Shepherd']


In [29]:
adj_list = ', '.join(f"'{adj}'" for adj in adjusters)
print(adj_list)

'CRISTAL WILLIAMS', 'KOLLEEN MCDONNOLD', 'Leah Shepherd'


In [14]:
auth_limits=[]
for index, row in claims_df.iterrows():
    lim_sql = '''
            SELECT authority
            FROM authority
            WHERE adjuster = '{adjuster}'
            AND effective_date <= '{paid_date}'
            ORDER BY effective_date DESC
            LIMIT 1;
            '''.format(adjuster = row['adjuster'], paid_date = row['paid_date'])
    adj_limit = mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_audit', lim_sql, 1, 0)
    auth_limits.append(adj_limit[0][0][0])

# add authority limit to each claim
claims_df.insert(6, "authority", auth_limits)
claims_df.head(5)

,claim_id,claim_nbr,paid_amount,paid_date,adjuster,MAX(s.status_id),authority
0,74736,2022033075741,752.37,2022-06-07,CRISTAL WILLIAMS,405896,1000.0
1,74821,2022041275826,1243.25,2022-06-06,Leah Shepherd,405858,1000.0
2,74850,2022041275855,930.03,2022-06-10,CRISTAL WILLIAMS,406079,1000.0
3,75177,2022052076182,5327.77,2022-06-09,Leah Shepherd,406008,10000.0
4,75236,2022052776241,1217.78,2022-06-10,Leah Shepherd,406102,10000.0


In [23]:
numOrPer = 'num'
numPer = 5

for adjuster in adjusters:
    # filter claims_df by adj_limits_df
    filtered_df = claims_df[(claims_df['adjuster'] == adjuster) &
                            (claims_df['paid_amount'] <= claims_df['authority'])]

    if numOrPer == 'num':
        factor = numPer
    else:
        perOfClaims = float(numPer)/100
        factor = filtered_df.shape[0] * perOfClaims

    # Random sample of results
    # If less than 5 then select all
    if filtered_df.shape[0] < 5:
        random_df = filtered_df
    else:
        if numOrPer == 'num':
            factor = numPer
        else:
            factor = max(5,filtered_df.shape[0] * float(numPer)/100)
        random_df = filtered_df.sample(n=factor, random_state=13)

    # Remove column 'MAX(s.status_id)'
    random_df.drop(columns=['MAX(s.status_id)'], inplace=True)
    # add random sample to audit_df
    audit_df = pd.concat([audit_df, random_df])

C:\Users\jbehler\AppData\Local\Temp\ipykernel_35324\1704952089.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_df.drop(columns=['MAX(s.status_id)'], inplace=True)


In [24]:
audit_df

,claim_id,claim_nbr,paid_amount,paid_date,adjuster,authority
11,75295,2022060676300,974.56,2022-06-06,CRISTAL WILLIAMS,1000.0
15,75302,2022060776307,70.05,2022-06-08,CRISTAL WILLIAMS,1000.0
16,75313,2022060876318,684.17,2022-06-10,CRISTAL WILLIAMS,1000.0
2,74850,2022041275855,930.03,2022-06-10,CRISTAL WILLIAMS,1000.0
12,75297,2022060676302,463.73,2022-06-06,CRISTAL WILLIAMS,1000.0
7,75284,2022060376289,229.63,2022-06-06,KOLLEEN MCDONNOLD,1000.0
20,75319,2022060876324,5434.08,2022-06-09,Leah Shepherd,10000.0
27,75335,2022061076340,992.92,2022-06-10,Leah Shepherd,10000.0
26,75330,2022060976335,747.24,2022-06-09,Leah Shepherd,10000.0
10,75294,2022060676299,951.37,2022-06-06,Leah Shepherd,1000.0


In [25]:
# convert columns list to string
audit_cols = ", ".join(audit_columns)

# insert into sql
for index, rows in audit_df.iterrows():
    audit_sql = '''INSERT INTO claims ({columns}) VALUES ({claimId}, "{claimNbr}", {paidAmt}, "{paidDate}", "{adj}", {auth});'''.format(columns=audit_cols,
                    claimId=rows['claim_id'], claimNbr=rows['claim_nbr'], paidAmt=rows['paid_amount'], paidDate=rows['paid_date'], adj=rows['adjuster'],
                    auth=rows['authority'])
    # print (audit_sql)
    # Add to claim_audit database
    mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_audit', audit_sql, 0, 1)


In [ ]:
# create claims to audit report
audit_list_sql = '''SELECT claim_nbr, paid_amount, paid_date, adjuster, authority, audit_amount, audit_date
                    FROM claims
                    WHERE audit_date IS NULL;'''
audit_list = mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_audit', audit_list_sql, 1, 0)
audit_list_df = pd.DataFrame(audit_list[0])
audit_list_df.columns = audit_list[1]
audit_list_df['audit_amount'] = audit_list_df['audit_amount'].astype(str).replace({'None':''})
audit_list_df['audit_date'] = audit_list_df['audit_date'].astype(str).replace({'None':''})
audit_list_df

In [ ]:
# create pdf report
audit_list_html = './temp/audit_list.html'
audit_list_df.to_html(audit_list_html)
audit_list_pdf = './reports/audit_list.pdf'
pdf.from_file(audit_list_html, audit_list_pdf)

In [ ]:
# Get user name
user = os.getlogin()
print (user)